This example is quite simple and does not have incredible results, but as you use more compute and use larger models, you can get better results.

Also, if you're willing to spend some money you could use the OpenAI api to use their advanced models.

In [113]:
!pip install transformers

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [111]:
from transformers import pipeline
import random

In [105]:
class Character:

    def __init__(self, name, health, power, backstory=""):
        self.name = name
        self.health = health
        self.power = power
        self.backstory = backstory

    def is_alive(self):
        return self.health > 0

    def attack(self, other):
        # Generate random damage between 70% and 130% of base power
        damage = int(self.power * random.uniform(0.7, 1.3))
        other.health -= damage
        print(f"\n{self.name} attacks {other.name} for {damage} damage!")

    def heal(self, amount):
        self.health += amount
        print(f"\n{self.name} heals for {amount} health!")

In [106]:
class Player(Character):

    def __init__(self, name, health, power, backstory=""):
        super().__init__(name, health, power, backstory)

    def input_action(self):
        action = input("Enter an action (attack/heal/talk/quit): ")
        return action

In [107]:
class Enemy(Character):

    def __init__(self, name, health, power, backstory=""):
        super().__init__(name, health, power, backstory)

In [36]:
# We can use much larger models that are better, but that would take some time
# Check out llama here: https://huggingface.co/meta-llama
# text_generator = pipeline("text-generation", model="gpt2")
text_generator = pipeline("text-generation", model="bigscience/bloom-560m")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [37]:
# question_answerer = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [97]:
# Generate a dynamic enemy name using the text generation model
enemy_name = text_generator("A powerful English enemy name: ", max_length=20, num_return_sequences=1)[0]['generated_text']

In [98]:
enemy_name

"A powerful English enemy name:  The Devil's Eye. The Devil's Eye is a powerful English enemy"

In [108]:
# Add backstory and environmental context
player_backstory = "A wandering martial artist seeking to restore peace to the land after his village was destroyed by dark forces."
enemy_backstory = "Once a noble knight, now cursed and twisted by dark magic, driven to madness by an insatiable hunger for power."
environment = "The battle takes place in the ruins of an ancient temple, its walls covered in mysterious runes and overgrown with vines. The air is thick with tension and the distant sound of thunder."

In [109]:
# Create instances of the player and enemy
player = Player("Kung Fu Panda", 100, 10, player_backstory)
enemy = Enemy("The Devil's Eye", 50, 5, enemy_backstory)

In [112]:
# Game loop
while True:
    # Display the current status
    print(f"\n{player.name}'s Health: {player.health}")
    print(f"{enemy.name}'s Health: {enemy.health}")

    # Get the player's action
    action = player.input_action()

    if action == "attack":
        # Player attacks the enemy
        player.attack(enemy)

        # Check if the enemy is defeated
        if not enemy.is_alive():
            print(f"\n{player.name} has defeated {enemy.name}!")
            break

        # Enemy attacks the player
        enemy.attack(player)

        # Check if the player is defeated
        if not player.is_alive():
            print(f"\n{player.name} has been defeated by {enemy.name}!")
            break

    elif action == "heal":
        # Player heals themselves
        player.heal(10)

    elif action == "talk":
        # Player talks to the enemy
        question = input("What do you want to ask the enemy? ")

        # Use the question-answering model to generate a response
        context = (f"Player {player.name} is fighting against enemy {enemy.name}.\n"
                   f"Player backstory: {player.backstory}\n"
                   f"Enemy backstory: {enemy.backstory}\n"
                   f"Environment: {environment}")
        result = question_answerer(question=question, context=context)

        print(f"\n{enemy.name} says: {result['answer']}")

    elif action == "quit":
        # Quit the game
        print(f"\n{player.name} has quit the game.")
        break

    else:
        print("\nInvalid action. Please choose attack, heal, talk, or quit.")


Kung Fu Panda's Health: 100
The Devil's Eye's Health: 50

Kung Fu Panda attacks The Devil's Eye for 9 damage!

The Devil's Eye attacks Kung Fu Panda for 4 damage!

Kung Fu Panda's Health: 96
The Devil's Eye's Health: 41

Kung Fu Panda attacks The Devil's Eye for 10 damage!

The Devil's Eye attacks Kung Fu Panda for 4 damage!

Kung Fu Panda's Health: 92
The Devil's Eye's Health: 31

Kung Fu Panda attacks The Devil's Eye for 9 damage!

The Devil's Eye attacks Kung Fu Panda for 4 damage!

Kung Fu Panda's Health: 88
The Devil's Eye's Health: 22

Kung Fu Panda attacks The Devil's Eye for 9 damage!

The Devil's Eye attacks Kung Fu Panda for 5 damage!

Kung Fu Panda's Health: 83
The Devil's Eye's Health: 13

Kung Fu Panda attacks The Devil's Eye for 7 damage!

The Devil's Eye attacks Kung Fu Panda for 4 damage!

Kung Fu Panda's Health: 79
The Devil's Eye's Health: 6

Kung Fu Panda heals for 10 health!

Kung Fu Panda's Health: 89
The Devil's Eye's Health: 6

Kung Fu Panda heals for 10 health!